<a href="https://colab.research.google.com/github/github123-alt/AnK_Internship/blob/main/Week_2_Day_2_at_AnK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install open3d
!pip install wget

In [ ]:
import numpy as np
import open3d as o3d

In [ ]:
input_path='../DATA/'
output_path='../RESULTS/'
data_name = 'sample_w_normals.xyz'
point_cloud=np.loadtxt(input_path+data_name, skiprows=1)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

In [ ]:
o3d.visualization.draw_plotly([pcd])

Buffered data was truncated after reaching the output size limit.

**BPA**

In [ ]:
distances=pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3*avg_dist

In [ ]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector([radius, radius*2]))

In [ ]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

In [ ]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

**Poisson**'**s** ***Reconstruction***

In [ ]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [ ]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

**Export and Visualization**

In [ ]:
o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply",dec_mesh)
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply",p_mesh_crop)

In [ ]:
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod=mesh.simplify_quadric_decimation(i)

        o3d.io.write_triangle_mesh(output_path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
        print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
my_lods = lod_mesh_export(dec_mesh, [100000,50000,10000,1000,100], ".ply", output_path)

In [ ]:
o3d.visualization.draw_plotly([my_lods[100000]])